## オーバーレイ解析を実行してみよう

In [3]:
%%time
#ArcGIS API for Python を使った場合
from arcgis.gis import GIS # GIS クラスを利用
from arcgis.features import summarize_data # summarize_data クラスを利用

# ArcGIS Online へログイン
gis=GIS("home") # GIS メソッドに引数 "home" を指定して実行

#石川県の交通事故データを取得
traffic=gis.content.search(query="石川県_交通事故データ2021",item_type="Feature Layer Collection")[0]
#石川県の 1km のメッシュを取得
mesh=gis.content.search(query="石川県メッシュ",item_type="Feature Layer Collection")[0]

#オーバーレイ解析:空間結合 を実行
apimake_join=summarize_data.join_features(target_layer=mesh,    # 石川県の 1km メッシュデータ
                                      join_layer= traffic,               # 石川県の交通事故データ
                                      spatial_relationship="intersects", # 二つのデータが重なっているとき
                                      output_name="石川県メッシュ_交通事故_PythonAPI",
                                     join_type="LEFT"                    # 結合を外部結合に
                                     )

{"cost": 4.72}


Wall time: 1min 9s


In [4]:
api_map=gis.map("日本")
api_map

MapView(layout=Layout(height='400px', width='100%'))

In [5]:
# 作成した空間結合レイヤーにレンダラーを設定しながら上のマップに描画
api_map.add_layer(apimake_join,
                 {
    "type":"FeatureLayer",    
    "renderer":"ClassedColorRenderer",# 等級色
    "field_name":"Join_Count",#分類するフィールド名
    "opacity":0.7#透過度を設定
  })

In [6]:
#交通事故のポイントも描画
api_map.add_layer(traffic)

In [7]:
%%time
# ArcPy を使った場合
import arcpy

#石川県の交通事故データの URL を取得
traffic_url = r"https://services3.arcgis.com/qcPpjnUOIagfSFbp/arcgis/rest/services/af18b8/FeatureServer/0"
#石川県の交通事故データを File GeoDatabase に格納
traffics_lyr=arcpy.conversion.ExportFeatures(traffic_url,arcpy.env.workspace+"\石川県_交通事故2021")

#石川県の 1km のメッシュの URL を取得
mesh_url = r"https://services3.arcgis.com/qcPpjnUOIagfSFbp/arcgis/rest/services/a1a1db/FeatureServer/0"
#石川県の 1km のメッシュを File GeoDatabase に格納
mesh_lyr=arcpy.conversion.ExportFeatures(mesh_url,arcpy.env.workspace+"\石川県1kmメッシュ")

#オーバーレイ解析:空間結合 を実行
arcpy.analysis.SpatialJoin(
    target_features=mesh_lyr,# 石川県の 1km メッシュデータ
    join_features=traffics_lyr,# 石川県の交通事故データ
    out_feature_class=arcpy.env.workspace+"\石川県メッシュ_交通事故_ArcPy",
    join_operation="JOIN_ONE_TO_ONE",#1 対 1 で解析を実行
    join_type="KEEP_ALL",# すべての値を保持
    match_option="INTERSECT"#二つのデータが重なっている 
)

Wall time: 17.9 s


<Result 'C:\\output\\Python\\Python.gdb\\石川県メッシュ_交通事故_ArcPy'>

In [8]:
# プロジェクト内で作成したマップオブジェクト上での表示を変更

# プロジェクト ファイルのオブジェクト取得
aprx = arcpy.mp.ArcGISProject("CURRENT")
# マップのオブジェクト取得
maps = aprx.listMaps()[0]
# レイヤーのオブジェクト一覧を取得
lyrs =maps.listLayers()
# レイヤーの一覧から 石川県メッシュ_交通事故_ArcPy を探してレンダラーを適用
for lyr in lyrs:
    if lyr.name == "石川県メッシュ_交通事故_ArcPy":
        sym = lyr.symbology
        sym.updateRenderer("GraduatedColorsRenderer") # 等級色
        sym.renderer.classificationField = "Join_Count"  #分類するフィールド名
        sym.renderer.classificationMethod = "Quantile" # 等比間隔
        sym.renderer.breakCount = 5 # クラス
        lyr.symbology = sym